In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
"""
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
"""

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [ ]:
#!pip install pytorch


In [ ]:
import torch
import torchvision.models as models
import torchvision

from torchvision import transforms,datasets
import cv2


In [ ]:
model = models.resnet18(pretrained=False)
model.fc=torch.nn.Linear(512,5)


In [ ]:
#torch.save(model.state_dict(), "./resnet18")


In [ ]:
train_dir="../input/aptos2019-blindness-detection/train_images/"
test_dir="../input/aptos2019-blindness-detection/test_images/"
df=pd.read_csv("../input/aptos2019-blindness-detection/train.csv")


In [ ]:
optimizer = torch.optim.Adam(model.parameters(),lr = 1e-3)
#loss = torch.nn.CrossEntropyLoss()
loss=torch.nn.MSELoss()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model=model.to(device)


In [ ]:
c=0
for i in df.values:
    optimizer.zero_grad()
    image_id,diagnosis=i
    image_path=os.path.join(train_dir,image_id+".png")
    img = cv2.imread(image_path)
    h,w,ch=img.shape
    img=transforms.ToTensor()(img).unsqueeze_(0)/256
    """
    delta=abs(h-w)
    if h>w:
        img=transforms.Pad(padding=[0,delta//2])(img)
    elif w>h:
        img=transforms.Pad(padding=[delta//2,0])(img)
    """
    img=img.to(device)
    preds=model.forward(img)
    true_tensor=[0.0,0.0,0.0,0.0,0.0]
    true_tensor[diagnosis]=1.0
    true_tensor=torch.Tensor(true_tensor).to(device)
    loss_value = loss(preds[0], true_tensor)
    loss_value.backward()
    optimizer.step()
    c+=1
    if c%20==0:
        print(c, loss_value)
    if c==500:
        break


In [ ]:
df=pd.read_csv("../input/aptos2019-blindness-detection/test.csv")


In [ ]:
df


In [ ]:
out=[]
c=0
for i in df.values:
    image_id=i[0]
    image_path=os.path.join(test_dir,image_id+".png")
    img = cv2.imread(image_path)
    h,w,ch=img.shape
    img=transforms.ToTensor()(img).unsqueeze_(0)/256
    """
    delta=abs(h-w)
    if h>w:
        img=transforms.Pad(padding=[0,delta//2])(img)
    elif w>h:
        img=transforms.Pad(padding=[delta//2,0])(img)
    """
    img=img.to(device)
    preds=model.forward(img)
    max_class=max(preds[0].tolist())
    max_index=preds[0].tolist().index(max_class)
    out.append([image_id,max_index])
    c+=1
    if c%50==0:
        print(c)


In [ ]:
df = pd.DataFrame(out)
df.columns=["id_code","diagnosis"]
df.to_csv('/kaggle/working/submission.csv', index=False)


In [ ]:
df.describe()
